In [99]:
import pandas as pd
import numpy as np

from datetime import datetime

import re

from utils import tag_mail, format_mail

In [142]:
df = pd.read_json(r'data/gmail_boetto_basic.txt', encoding = 'utf-8')

In [143]:
df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='doctolib']

,snippet,date,headers,body,cat
1228,Doctolib RDV confirmé Clinique Ostéopathique d...,2018-11-21 16:45:29,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib
1869,Doctolib Rappel : vous avez un rendez-vous dem...,2017-01-01 14:25:31,"[{'name': 'From', 'value': 'Doctolib <julia@do...",DoctolibVotre rendez-vousVotre rendez-vous \n...,doctolib
1870,Doctolib Rappel : vous avez un rendez-vous le ...,2016-12-26 14:27:32,"[{'name': 'From', 'value': 'Doctolib <julia@do...",DoctolibVotre rendez-vousVotre rendez-vous \n...,doctolib
1875,Doctolib Votre rendez-vous du lundi 02 janvier...,2016-12-22 10:58:22,"[{'name': 'From', 'value': 'Doctolib <julia@do...",DoctolibVotre rendez-vousVotre rendez-vous \n...,doctolib


In [150]:
import locale
locale.setlocale(locale.LC_TIME, "fr_FR")

mails_doc = df.loc[df.cat == 'doctolib']
mails_doc = mails_doc.loc[mails_doc.snippet.str.contains("confirmé")]

reg_weekdays = re.compile(r' Lundi|Mardi|Mercredi|Jeudi|Vendredi|Samedi|Dimanche')

mails_doc['appointment'] = mails_doc.snippet.apply(lambda x: reg_weekdays.split(x.split('confirmé ')[1])[0] )

def get_date(x):
    tmp = x['snippet']
    tmp = tmp.split(x['appointment'])[1]
    tmp = tmp.split(' DÉPLACER')[0]
    try:
        date = datetime.strptime(tmp, '%A %d %B à %Hh%M')
        date = date.replace(year = x['date'].year)
    except:
        date = None
    return date
    
mails_doc['cons_date'] = mails_doc[['snippet', 'date', 'appointment']].apply(get_date, axis = 1) 

mails_doc = mails_doc.loc[mails_doc['cons_date'].isna() == False]

In [145]:
mails_doc

,snippet,date,headers,body,cat,docteur,cons_date
1228,Doctolib RDV confirmé Clinique Ostéopathique d...,2018-11-21 16:45:29,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib,Clinique Ostéopathique de Paris Ostéopathe,2018-11-24 11:00:00


In [121]:
def get_date(x):
    tmp = x['snippet']
    tmp = tmp.split(x['docteur'])[1]
    tmp = tmp.split(' DÉPLACER')[0]
    
    date = datetime.strptime(tmp, '%A %d %B à %Hh%M')
    date = date.replace(year = x['date'].year)
    return date
    
mails_doc[['snippet', 'date', 'docteur']].apply(get_date, axis = 1) 

155   2020-01-09 18:45:00
399   2019-10-16 09:15:00
418   2019-10-09 11:15:00
dtype: datetime64[ns]

In [146]:
import locale

In [147]:
locale.setlocale('fr_FR')

TypeError: an integer is required (got type str)

In [ ]:
import locale
locale.setlocale(locale.LC_TIME, "fr_FR")